# Resumo de textos com arquivos pequenos com o Anthropic Claude

> *Este caderno deve funcionar bem com o kernel **`Data Science 3.0`** no SageMaker Studio*

## Visão geral

Neste exemplo, você vai ingerir uma pequena quantidade de dados (Dados de string) diretamente na API do Amazon Bedrock (pelo modelo Anthropic Claude) e dar a ela a instrução de resumir o respectivo texto.

### Arquitetura

![](./images/41-text-simple-1.png)

Nesta arquitetura:

1. Um pequeno trecho de texto (ou arquivo pequeno) é carregado
1. Um modelo de base processa os dados de entrada
1. O modelo retorna uma resposta com o resumo do texto ingerido

### Caso de uso

Esta abordagem pode ser usada para resumir transcrições de chamadas, transcrições de reuniões, livros, artigos, postagens do blog e outros conteúdos relevantes.

### Desafios

Esta abordagem pode ser usada quando o texto ou arquivo de entrada cabe no comprimento do contexto do modelo. No caderno `02.long-text-summarization-titan.ipynb`, veremos uma abordagem para lidar com o desafio de quando os usuários têm documentos grandes que excedem o limite de tokens.

In [ ]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from labutils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

## Como resumir um texto curto com o boto3
 
Para aprender a detalhar a solicitação da API ao Amazon Bedrock, este caderno mostra como criar uma solicitação da API e enviá-la pelo Boto3, em vez de usar o LangChain, que fornece uma API mais simples incluindo a operação Boto3. 

### Sintaxe da solicitação da InvokeModel no Boto3


Usamos a API `InvokeModel` para enviar uma solicitação ao modelo de base. Aqui está um exemplo de solicitação da API para envio de textos ao Anthropic Claude. Os parâmetros de inferência em `textGenerationConfig` dependem do modelo que você está prestes a usar. Os parâmetros de inferência do Anthropic Claude são:

- **temperature** ajusta o grau de aleatoriedade na geração. Temperaturas menores significam gerações menos aleatórias.
- **top_p** menor do que um mantém apenas o menor conjunto dos tokens mais prováveis com probabilidades que somam top_p ou superior para geração.
- **top_k** pode ser usado para reduzir a repetitividade dos tokens gerados. Quanto maior o valor, mais rígida é a penalidade aprovada aos tokens previamente apresentados, é proporcional a quantas vezes eles já apareceram no prompt ou na geração anterior.
- **max_tokens_to_sample** é o número máximo de tokens a serem gerados. Não é garantido que a respostas preencherão o comprimento máximo desejado.
- **stop_sequences** são sequências em que a API vai parar de gerar mais tokens. O texto retornado não conterá a sequência de interrupção.

```python
response = bedrock.invoke_model(body=
                                {"prompt":"this is where you place your input text",
                                 "max_tokens_to_sample":4096,
                                 "temperature":0.5,
                                 "top_k":250,
                                 "top_p":0.5,
                                 "stop_sequences":[]
                                },
                                modelId="anthropic.claude-v2", 
                                accept=accept, 
                                contentType=contentType)

```

### Criar um prompt com texto a ser resumido

Neste caderno, você pode usar qualquer texto curto cujos tokens sejam menores do que o token máximo de um modelo de base. Como exemplo de um texto curto, vamos pegar um parágrafo de uma [publicação do blog da AWS](https://aws.amazon.com/jp/blogs/machine-learning/announcing-new-tools-for-building-with-generative-ai-on-aws/) sobre o anúncio do Amazon Bedrock.

O prompt começa com uma instrução `Forneça um resumo do seguinte texto`. 

In [ ]:
prompt = """

Human: Please provide a summary of the following text.
<text>
AWS took all of that feedback from customers, and today we are excited to announce Amazon Bedrock, \
a new service that makes FMs from AI21 Labs, Anthropic, Stability AI, and Amazon accessible via an API. \
Bedrock is the easiest way for customers to build and scale generative AI-based applications using FMs, \
democratizing access for all builders. Bedrock will offer the ability to access a range of powerful FMs \
for text and images—including Amazons Titan FMs, which consist of two new LLMs we’re also announcing \
today—through a scalable, reliable, and secure AWS managed service. With Bedrock’s serverless experience, \
customers can easily find the right model for what they’re trying to get done, get started quickly, privately \
customize FMs with their own data, and easily integrate and deploy them into their applications using the AWS \
tools and capabilities they are familiar with, without having to manage any infrastructure (including integrations \
with Amazon SageMaker ML features like Experiments to test different models and Pipelines to manage their FMs at scale).
</text>

Assistant:"""

## Criar corpo da solicitação com prompt e parâmetros de inferência 

Após a sintaxe da solicitação de `invoke_model`, você cria o corpo da solicitação com o prompt e os parâmetros de inferência acima.

In [ ]:
body = json.dumps({"prompt": prompt,
                 "max_tokens_to_sample":4096,
                 "temperature":0.5,
                 "top_k":250,
                 "top_p":0.5,
                 "stop_sequences":[]
                  }) 

## Invocar modelo de base pelo Boto3

A solicitação da API é enviada ao Amazon Bedrock com a especificação dos parâmetros de solicitação `modelId`, `accept` e `contentType`. Seguindo o prompt, o modelo de base no Amazon Bedrock resume o texto.

In [ ]:
modelId = 'anthropic.claude-v2' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print_ww(response_body.get('completion'))

Acima, o serviço do Bedrock gera todo o resumo para o prompt em um único resultado, o que pode demorar se o resultado contiver uma grande quantidade de tokens. 

Abaixo nós vemos a opção de como usar o Bedrock para transmitir o resultado de forma que o usuário possa começar a consumi-lo conforme é gerado pelo modelo. Para tal, o Bedrock oferece suporte a API `invoke_model_with_response_stream` fornecendo `ResponseStream` que transmite esse resultado na forma de blocos.

In [ ]:
response = boto3_bedrock.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = list(stream)
output

Em vez de gerar toda o resultado, o Bedrock envia blocos menores do modelo. Isso pode ser exibido de uma forma consumível também.

In [ ]:
from IPython.display import display_markdown,Markdown,clear_output

In [ ]:
response = boto3_bedrock.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = []
i = 1
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            text = chunk_obj['completion']
            clear_output(wait=True)
            output.append(text)
            display_markdown(Markdown(''.join(output)))
            i+=1

## Conclusão
Você adquiriu experiência com o uso do SDK `boto3`, que oferece uma exposição básica à API do Amazon Bedrock. Ao usar esta API, você viu o caso de uso da geração de um resumo das notícias do AWS sobre o Amazon Bedrock de duas formas diferentes: o resultado inteiro e a transmissão da geração do resultado.

### Conclusões
- Adaptar este caderno para testar diferentes modelos disponíveis através do Amazon Bedrock, como modelos Amazon Titan e AI21 Labs Jurassic.
- Alterar os prompts para seu caso de uso específico e avaliar o resultado de diferentes modelos.
- Experimentar com o comprimento do token para entender a latência e a responsividade do serviço.
- Aplicar diferentes princípios de engenharia de prompt para obter resultados melhores.

## Agradecemos sua atenção